In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/common-surnames-by-country.csv'

import pandas as pd
df = pd.read_csv(path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Country,Rank,Index,Name Group,Romanized Name,Count
0,AM,1.0,1,AM-1,Grigoryan,NaN
1,AM,2.0,2,AM-2,Harutyunyan,NaN
2,AM,3.0,3,AM-3,Sargsyan,NaN
3,AM,4.0,4,AM-4,Hovhannisyan,NaN
4,AM,5.0,5,AM-5,Khachatryan,NaN
...,...,...,...,...,...,...
2571,SR,9.0,9,SR-9,van Dijk,NaN
2572,SR,10.0,10,SR-10,Tjin,NaN
2573,SR,11.0,11,SR-11,Kluivert,NaN
2574,SR,12.0,12,SR-12,Semil,NaN


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ['Country', 'Romanized Name']


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

   Country  Rank  Index Name Group Romanized Name  Count
0        A   1.0      1       AM-1              G    NaN
1        A   2.0      2       AM-2              H    NaN
2        A   3.0      3       AM-3              S    NaN
3        A   4.0      4       AM-4              H    NaN
4        A   5.0      5       AM-5              K    NaN
5        A   6.0      6       AM-6              H    NaN
6        A   7.0      7       AM-7              P    NaN
7        A   8.0      8       AM-8              V    NaN
8        A   9.0      9       AM-9              G    NaN
9        A  10.0     10      AM-10              K    NaN
10       A  11.0     11      AM-11              M    NaN
11       A  12.0     12      AM-12              G    NaN
12       A  13.0     13      AM-13              M    NaN
13       A  14.0     14      AM-14              A    NaN
14       A  15.0     15      AM-15              P    NaN
15       A  16.0     16      AM-16              M    NaN
16       A  17.0     17      AM

<ipython-input-7-1033100105>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ Country: 2526 full names, 50 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
❌ Romanized Name: 2524 full names, 44 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
